# Marketing A/B Test - Business Impact Analysis

**Author**: Analytics Team  
**Date**: November 2025  
**Version**: 1.0

---

## Executive Summary

This notebook calculates comprehensive business metrics to assess the financial impact of the advertising campaign. These metrics enable data-driven decision-making about campaign deployment and scaling.

## Business Metrics

1. **Incremental Conversions**: Additional conversions directly attributed to the advertising campaign
2. **Cost per Incremental Acquisition (CPA)**: Cost to acquire one incremental conversion through advertising
3. **Return on Ad Spend (ROAS)**: Revenue generated per dollar spent on advertising
4. **Return on Investment (ROI)**: Net profit relative to investment
5. **Break-Even Analysis**: Minimum performance threshold needed to justify campaign costs
6. **Scaling Projections**: Expected impact at different user volumes

## Business Assumptions

- **Value per Conversion**: $100 (customizable)
- **Cost per Impression**: $0.01 (customizable)
- These assumptions can be adjusted based on actual business parameters


In [1]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

# Load data
df = pd.read_csv('marketing_AB.csv')

# Try to load statistical results if available
try:
    with open('frequentist_results.json', 'r') as f:
        frequentist_results = json.load(f)
except FileNotFoundError:
    frequentist_results = None

try:
    with open('bayesian_results.json', 'r') as f:
        bayesian_results = json.load(f)
except FileNotFoundError:
    bayesian_results = None

print("Data and results loaded successfully!")


FileNotFoundError: [Errno 2] No such file or directory: 'marketing_AB.csv'

## 1. Incremental Conversions

### Formula

$$\text{Incremental Conversions} = n_{\text{ad}} \times (CR_{\text{ad}} - CR_{\text{control}})$$

This represents the number of conversions that can be directly attributed to the advertising campaign, beyond what would have occurred naturally.


In [ ]:
# Separate groups
ad_group = df[df['test group'] == 'ad']
psa_group = df[df['test group'] == 'psa']

# Basic metrics
n_ad = len(ad_group)
n_psa = len(psa_group)
ad_conversions = ad_group['converted'].sum()
psa_conversions = psa_group['converted'].sum()

cr_ad = ad_conversions / n_ad
cr_psa = psa_conversions / n_psa

# Incremental conversions
incremental_conversions = ad_conversions - (n_ad * cr_psa)
incremental_conversion_rate = cr_ad - cr_psa

print("INCREMENTAL CONVERSIONS")
print("="*60)
print(f"Ad Group:")
print(f"  Users: {n_ad:,}")
print(f"  Conversions: {ad_conversions:,}")
print(f"  Conversion Rate: {cr_ad:.4%}")

print(f"\nPSA Group (Control):")
print(f"  Users: {n_psa:,}")
print(f"  Conversions: {psa_conversions:,}")
print(f"  Conversion Rate: {cr_psa:.4%}")

print(f"\nIncremental Impact:")
print(f"  Incremental Conversion Rate: {incremental_conversion_rate:.6f} ({incremental_conversion_rate*100:.4f}%)")
print(f"  Incremental Conversions: {incremental_conversions:,.2f}")


## 2. Cost per Incremental Acquisition (CPA)

### Formula

$$\text{CPA} = \frac{\text{Total Campaign Cost}}{\text{Incremental Conversions}}$$

This metric tells us how much it costs to acquire one incremental conversion through advertising.


In [ ]:
# Business assumptions
cost_per_impression = 0.01  # Cost per ad shown
total_ad_impressions = ad_group['total ads'].sum()
total_campaign_cost = total_ad_impressions * cost_per_impression

# Cost per conversion
cost_per_conversion_ad = total_campaign_cost / ad_conversions if ad_conversions > 0 else float('inf')

# Cost per incremental acquisition (CPA)
cost_per_incremental_acquisition = total_campaign_cost / incremental_conversions if incremental_conversions > 0 else float('inf')

print("COST ANALYSIS")
print("="*60)
print(f"Total Ad Impressions: {total_ad_impressions:,}")
print(f"Cost per Impression: ${cost_per_impression:.4f}")
print(f"Total Campaign Cost: ${total_campaign_cost:,.2f}")
print(f"\nCost per Conversion (Ad Group): ${cost_per_conversion_ad:.2f}")
print(f"Cost per Incremental Acquisition (CPA): ${cost_per_incremental_acquisition:.2f}")


## 3. Return on Ad Spend (ROAS)

### Formula

$$\text{ROAS} = \frac{\text{Revenue}}{\text{Ad Spend}}$$

### ROI Formula

$$\text{ROI} = \frac{\text{Revenue} - \text{Cost}}{\text{Cost}} = \text{ROAS} - 1$$

### Interpretation

- **ROAS > 1.0**: Campaign is profitable
- **ROAS = 1.0**: Break-even
- **ROAS < 1.0**: Campaign is losing money


In [ ]:
# Visualization: Business Impact Metrics
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# ROAS Comparison
metrics = ['Total\nRevenue', 'Incremental\nRevenue']
roas_values = [roas_total, roas_incremental]
colors = ['#3B82F6', '#10B981']
bars = axes[0].bar(metrics, roas_values, color=colors, alpha=0.8, edgecolor='black', linewidth=1.5)
axes[0].axhline(y=1.0, color='red', linestyle='--', linewidth=2, label='Break-Even (ROAS=1.0)')
axes[0].set_ylabel('ROAS (x)', fontsize=12, fontweight='bold')
axes[0].set_title('Return on Ad Spend (ROAS)', fontsize=14, fontweight='bold', pad=20)
axes[0].legend(fontsize=10)
axes[0].grid(axis='y', alpha=0.3, linestyle='--')
for bar, val in zip(bars, roas_values):
    axes[0].text(bar.get_x() + bar.get_width()/2, val + 0.1, f'{val:.2f}x', 
                ha='center', fontweight='bold', fontsize=11)

# Revenue vs Cost
categories = ['Total\nRevenue', 'Incremental\nRevenue', 'Campaign\nCost']
values = [total_revenue_ad, incremental_revenue, total_campaign_cost]
colors_rev = ['#10B981', '#34D399', '#EF4444']
bars2 = axes[1].bar(categories, values, color=colors_rev, alpha=0.8, edgecolor='black', linewidth=1.5)
axes[1].set_ylabel('Amount ($)', fontsize=12, fontweight='bold')
axes[1].set_title('Revenue vs Campaign Cost', fontsize=14, fontweight='bold', pad=20)
axes[1].grid(axis='y', alpha=0.3, linestyle='--')
for bar, val in zip(bars2, values):
    axes[1].text(bar.get_x() + bar.get_width()/2, val + max(values)*0.02, 
                f'${val:,.0f}', ha='center', fontweight='bold', fontsize=10)

# Scaling Projections
scaling_users = [10, 50, 100, 500, 1000]  # in thousands
scaling_revenue = [u * 1000 * incremental_conversion_rate * value_per_conversion for u in scaling_users]
scaling_cost = [u * 1000 * (total_ad_impressions / n_ad) * cost_per_impression for u in scaling_users]
x_pos = np.arange(len(scaling_users))
width = 0.35
axes[2].bar(x_pos - width/2, [r/1000 for r in scaling_revenue], width, label='Revenue (K$)', 
           color='#10B981', alpha=0.8, edgecolor='black', linewidth=1.5)
axes[2].bar(x_pos + width/2, [c/1000 for c in scaling_cost], width, label='Cost (K$)', 
           color='#EF4444', alpha=0.8, alpha=0.8, edgecolor='black', linewidth=1.5)
axes[2].set_xlabel('Users (thousands)', fontsize=12, fontweight='bold')
axes[2].set_ylabel('Amount (K$)', fontsize=12, fontweight='bold')
axes[2].set_title('Scaling Projections', fontsize=14, fontweight='bold', pad=20)
axes[2].set_xticks(x_pos)
axes[2].set_xticklabels([f'{u}K' for u in scaling_users])
axes[2].legend(fontsize=11)
axes[2].grid(axis='y', alpha=0.3, linestyle='--')

plt.tight_layout()
plt.show()


In [ ]:
# Revenue calculations
value_per_conversion = 100
total_revenue_ad = ad_conversions * value_per_conversion
incremental_revenue = incremental_conversions * value_per_conversion

# ROAS
roas_total = total_revenue_ad / total_campaign_cost if total_campaign_cost > 0 else float('inf')
roas_incremental = incremental_revenue / total_campaign_cost if total_campaign_cost > 0 else float('inf')

# ROI
roi_total = (total_revenue_ad - total_campaign_cost) / total_campaign_cost if total_campaign_cost > 0 else float('inf')
roi_incremental = (incremental_revenue - total_campaign_cost) / total_campaign_cost if total_campaign_cost > 0 else float('inf')

print("RETURN ON AD SPEND (ROAS)")
print("="*60)
print(f"Value per conversion: ${value_per_conversion}")
print(f"Total Revenue (Ad Group): ${total_revenue_ad:,.2f}")
print(f"Incremental Revenue: ${incremental_revenue:,.2f}")
print(f"\nROAS Metrics:")
print(f"  ROAS (Total Revenue): {roas_total:.2f}x")
print(f"    For every $1 spent, generated ${roas_total:.2f} in revenue")
print(f"  ROAS (Incremental Revenue): {roas_incremental:.2f}x")
print(f"    For every $1 spent, generated ${roas_incremental:.2f} in incremental revenue")

print(f"\nROI Metrics:")
print(f"  ROI (Total): {roi_total:.2%}")
print(f"  ROI (Incremental): {roi_incremental:.2%}")

if roas_incremental > 1:
    print(f"\n✅ Campaign is profitable (ROAS > 1.0)")
elif roas_incremental > 0.5:
    print(f"\n⚠️  Campaign is marginally profitable")
else:
    print(f"\n❌ Campaign is not profitable (ROAS < 0.5)")


## 4. Break-Even Analysis

### Break-Even Point

The break-even point is where incremental revenue equals campaign cost:

$$\text{Incremental Conversions} \times \text{Value per Conversion} = \text{Total Campaign Cost}$$

Solving for incremental conversions:

$$\text{Break-Even Conversions} = \frac{\text{Total Campaign Cost}}{\text{Value per Conversion}}$$

### Break-Even Conversion Rate

$$\text{Break-Even CR Lift} = \frac{\text{Break-Even Conversions}}{n_{\text{ad}}}$$


In [ ]:
# Break-even point
break_even_conversions = total_campaign_cost / value_per_conversion if value_per_conversion > 0 else float('inf')
break_even_conversion_rate = break_even_conversions / n_ad

current_incremental = incremental_conversions
conversion_rate_needed = break_even_conversion_rate

print("BREAK-EVEN ANALYSIS")
print("="*60)
print(f"Break-Even Incremental Conversions: {break_even_conversions:,.2f}")
print(f"Current Incremental Conversions: {current_incremental:,.2f}")
print(f"Break-Even Conversion Rate Lift: {conversion_rate_needed:.6f} ({conversion_rate_needed*100:.4f}%)")
print(f"Current Conversion Rate Lift: {incremental_conversion_rate:.6f} ({incremental_conversion_rate*100:.4f}%)")

if current_incremental > break_even_conversions:
    margin = current_incremental - break_even_conversions
    print(f"\n✅ Campaign exceeds break-even by {margin:,.2f} conversions")
else:
    shortfall = break_even_conversions - current_incremental
    print(f"\n⚠️  Campaign needs {shortfall:,.2f} more incremental conversions to break even")


## 5. Scaling Projections

We can project the expected impact at different user volumes, assuming the same conversion rate lift.

### Scaling Formula

For $n$ users:

$$\text{Projected Incremental Conversions} = n \times (CR_{\text{ad}} - CR_{\text{psa}})$$

$$\text{Projected Revenue} = \text{Projected Incremental Conversions} \times \text{Value per Conversion}$$

$$\text{Projected Cost} = n \times \frac{\text{Total Impressions}}{n_{\text{ad}}} \times \text{Cost per Impression}$$


## Summary & Business Recommendations

### Financial Impact Summary
- **Incremental Conversions**: {incremental_conversions:,.2f}
- **Incremental Revenue**: ${incremental_revenue:,.2f}
- **Total Campaign Cost**: ${total_campaign_cost:,.2f}
- **ROAS (Incremental)**: {roas_incremental:.2f}x
- **ROI (Incremental)**: {roi_incremental:.2%}

### Break-Even Status
- **Break-Even Conversions**: {break_even_conversions:,.2f}
- **Current Incremental**: {current_incremental:,.2f}
- **Status**: {break_even_status}

### Scaling Potential
The analysis shows projected impact at various scales, demonstrating the scalability of the campaign.

### Recommendations
Based on the business impact analysis:
- {business_recommendation}


In [ ]:
# Business impact summary
break_even_status = f"Exceeds break-even by {current_incremental - break_even_conversions:,.2f} conversions" if current_incremental > break_even_conversions else f"Needs {break_even_conversions - current_incremental:,.2f} more conversions to break even"

if roas_incremental > 1.5:
    business_recommendation = "✅ Strong ROI - Recommend full campaign deployment and aggressive scaling"
elif roas_incremental > 1.0:
    business_recommendation = "✅ Profitable - Recommend campaign deployment with monitoring"
elif roas_incremental > 0.5:
    business_recommendation = "⚠️ Marginally profitable - Consider optimization before scaling"
else:
    business_recommendation = "❌ Not profitable - Recommend campaign redesign or discontinuation"

print("\n" + "="*80)
print("BUSINESS IMPACT SUMMARY")
print("="*80)
print(f"\nFinancial Metrics:")
print(f"  Incremental Conversions: {incremental_conversions:,.2f}")
print(f"  Incremental Revenue: ${incremental_revenue:,.2f}")
print(f"  Total Campaign Cost: ${total_campaign_cost:,.2f}")
print(f"  ROAS (Incremental): {roas_incremental:.2f}x")
print(f"  ROI (Incremental): {roi_incremental:.2%}")

print(f"\nBreak-Even Analysis:")
print(f"  Break-Even Conversions: {break_even_conversions:,.2f}")
print(f"  Current Incremental: {current_incremental:,.2f}")
print(f"  Status: {break_even_status}")

print(f"\nRecommendation:")
print(f"  {business_recommendation}")

# Save results
summary = {
    'ad_group_size': n_ad,
    'psa_group_size': n_psa,
    'ad_conversions': ad_conversions,
    'psa_conversions': psa_conversions,
    'ad_conversion_rate': cr_ad,
    'psa_conversion_rate': cr_psa,
    'incremental_conversions': incremental_conversions,
    'incremental_conversion_rate': incremental_conversion_rate,
    'value_per_conversion': value_per_conversion,
    'total_campaign_cost': total_campaign_cost,
    'total_revenue_ad': total_revenue_ad,
    'incremental_revenue': incremental_revenue,
    'cost_per_incremental_acquisition': cost_per_incremental_acquisition,
    'roas_total': roas_total,
    'roas_incremental': roas_incremental,
    'roi_total': roi_total,
    'roi_incremental': roi_incremental,
    'break_even_conversions': break_even_conversions,
    'is_profitable': roas_incremental > 1.0
}

with open('business_impact_results.json', 'w') as f:
    json.dump({k: float(v) if isinstance(v, (np.integer, np.floating)) else v 
              for k, v in summary.items()}, f, indent=2)

print("\n✅ Results saved to 'business_impact_results.json'")
print("="*80)


In [ ]:
# Projections for different user volumes
scaling_scenarios = [10000, 50000, 100000, 500000, 1000000]

print("SCALING PROJECTIONS")
print("="*80)
print(f"{'Users':>12} {'Incremental Conv.':>20} {'Incremental Revenue':>25} {'Campaign Cost':>20} {'ROAS':>10}")
print("-" * 95)

for n_users in scaling_scenarios:
    projected_incremental = n_users * incremental_conversion_rate
    projected_revenue = projected_incremental * value_per_conversion
    # Assume same cost per impression, scale impressions proportionally
    impressions_per_user = total_ad_impressions / n_ad
    projected_impressions = n_users * impressions_per_user
    projected_cost = projected_impressions * cost_per_impression
    projected_roas = projected_revenue / projected_cost if projected_cost > 0 else float('inf')
    
    print(f"{n_users:>12,} {projected_incremental:>20,.0f} ${projected_revenue:>24,.2f} ${projected_cost:>19,.2f} {projected_roas:>10.2f}x")
